In [1]:
## 기본 library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
## model library
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from keras import models, layers, optimizers
## helper library
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from bayes_opt import BayesianOptimization

from sklearn.datasets import load_breast_cancer
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')


d = load_breast_cancer()

data = pd.DataFrame(d.data, columns=d.feature_names)
target = pd.DataFrame(d.target)



Using TensorFlow backend.


In [2]:
np.random.seed(0)

In [3]:
std = StandardScaler()
for c in data.columns:
    data[c] = std.fit_transform(data[c].values.reshape(-1, 1))
    
train_X, test_X, train_y, test_y = train_test_split(data, target, test_size=0.3, random_state=0, shuffle=True)

In [4]:
train_X = train_X.reset_index().drop('index', axis=1)
train_y = train_y.reset_index().drop('index', axis=1)
test_X = test_X.reset_index().drop('index', axis=1)
test_y = test_y.reset_index().drop('index', axis=1)

In [5]:
# km = KMeans(n_clusters=5).fit(train_X)
# cl = pd.get_dummies(km.labels_, prefix='kmean')
# train_X = pd.concat([train_X, cl], axis=1)

# # km = KMeans(n_clusters=5).fit(test_X)
# # cl = pd.get_dummies(km.labels_, prefix='kmean')
# # test_X = pd.concat([test_X, cl], axis=1)

# cl = pd.get_dummies(km.predict(test_X), prefix='kmean')
# test_X = pd.concat([test_X, cl], axis=1)

In [5]:
km = KMeans(n_clusters=5).fit(train_X)
tr_cl = pd.get_dummies(km.labels_, prefix='kmean')

te_cl = pd.get_dummies(km.predict(test_X), prefix='kmean')

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [7]:
lr = LogisticRegression(penalty='l2', random_state=0)

In [8]:
lr.fit(train_X, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
cross_val_score(lr, train_X, train_y, cv=kf)

array([0.9875    , 0.975     , 0.95      , 0.98734177, 0.98734177])

In [10]:
accuracy_score(test_y, lr.predict(test_X))

0.9766081871345029

In [35]:
K.clear_session()
input_layer = layers.Input(shape=((train_X.shape[1:])))

l1 = layers.Dense(32 ,activation='relu')(input_layer)
l1 = layers.Dropout(0.3)(l1)
l2 = layers.Dense(32, activation='relu')(l1)

out_layer = layers.Dense(1, activation='sigmoid')(l2)

model = models.Model(input_layer, out_layer)

model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['acc'])

In [36]:
es = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

In [37]:
model.fit(train_X, train_y, 
         epochs=300,
         validation_split=0.3,
         callbacks=[es]
         )

Train on 278 samples, validate on 120 samples
Epoch 1/300
278/278 [==============================] - 0s 613us/step - loss: 0.7110 - acc: 0.4892 - val_loss: 0.5652 - val_acc: 0.7667
Epoch 2/300
278/278 [==============================] - 0s 125us/step - loss: 0.5352 - acc: 0.7914 - val_loss: 0.4371 - val_acc: 0.9250
Epoch 3/300
278/278 [==============================] - 0s 126us/step - loss: 0.4270 - acc: 0.8849 - val_loss: 0.3469 - val_acc: 0.9417
Epoch 4/300
278/278 [==============================] - 0s 122us/step - loss: 0.3461 - acc: 0.9353 - val_loss: 0.2770 - val_acc: 0.9500
Epoch 5/300
278/278 [==============================] - 0s 129us/step - loss: 0.2764 - acc: 0.9281 - val_loss: 0.2245 - val_acc: 0.9500
Epoch 6/300
278/278 [==============================] - 0s 118us/step - loss: 0.2312 - acc: 0.9496 - val_loss: 0.1851 - val_acc: 0.9583
Epoch 7/300
278/278 [==============================] - 0s 122us/step - loss: 0.2110 - acc: 0.9460 - val_loss: 0.1555 - val_acc: 0.9750
Epoch 8/3

Epoch 61/300
278/278 [==============================] - 0s 169us/step - loss: 0.0313 - acc: 0.9928 - val_loss: 0.0331 - val_acc: 0.9917
Epoch 62/300
278/278 [==============================] - 0s 176us/step - loss: 0.0319 - acc: 0.9892 - val_loss: 0.0328 - val_acc: 0.9917
Epoch 63/300
278/278 [==============================] - 0s 179us/step - loss: 0.0313 - acc: 0.9892 - val_loss: 0.0317 - val_acc: 0.9917
Epoch 64/300
278/278 [==============================] - 0s 179us/step - loss: 0.0359 - acc: 0.9892 - val_loss: 0.0312 - val_acc: 0.9917
Epoch 65/300
278/278 [==============================] - 0s 165us/step - loss: 0.0255 - acc: 0.9892 - val_loss: 0.0312 - val_acc: 0.9917
Epoch 66/300
278/278 [==============================] - 0s 140us/step - loss: 0.0239 - acc: 0.9964 - val_loss: 0.0318 - val_acc: 0.9917
Epoch 67/300
278/278 [==============================] - 0s 133us/step - loss: 0.0318 - acc: 0.9928 - val_loss: 0.0318 - val_acc: 0.9917
Epoch 68/300
278/278 [==========================

278/278 [==============================] - 0s 129us/step - loss: 0.0151 - acc: 0.9928 - val_loss: 0.0308 - val_acc: 0.9917
Epoch 122/300
278/278 [==============================] - 0s 129us/step - loss: 0.0101 - acc: 1.0000 - val_loss: 0.0302 - val_acc: 0.9917
Epoch 123/300
278/278 [==============================] - 0s 126us/step - loss: 0.0126 - acc: 0.9964 - val_loss: 0.0296 - val_acc: 0.9917
Epoch 124/300
278/278 [==============================] - 0s 117us/step - loss: 0.0105 - acc: 0.9964 - val_loss: 0.0292 - val_acc: 0.9917
Epoch 125/300
278/278 [==============================] - 0s 122us/step - loss: 0.0242 - acc: 0.9964 - val_loss: 0.0297 - val_acc: 0.9917
Epoch 126/300
278/278 [==============================] - 0s 126us/step - loss: 0.0180 - acc: 0.9928 - val_loss: 0.0297 - val_acc: 0.9917
Epoch 127/300
278/278 [==============================] - 0s 125us/step - loss: 0.0197 - acc: 0.9928 - val_loss: 0.0299 - val_acc: 0.9917


In [38]:
model.evaluate(test_X, test_y)

171/171 [==============================] - 0s 70us/step


[0.13185035429992958, 0.9649122953414917]

In [21]:
svc = SVC(kernel='rbf', random_state=0).fit(train_X, train_y)

In [22]:
np.mean(cross_val_score(svc, train_X, train_y, cv=10))

0.9800000000000001